In [108]:
from network.srcnn import srcnn

import numpy
import math
import cv2

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [109]:
IMG_NAME = "./imageTrain/yayoi_first_030.png"

#FILEPATH_WEIGHT = "./preTrained/"
#FILENAME_WEIGHT = "srcnn.h5"

FILEPATH_WEIGHT = ""
FILENAME_WEIGHT = "yayoi_4_2018-07-13_092157.h5"

OPENCV_NAME = "opencv.bmp"
SRCNN_NAME = "srcnn.bmp"

INTERPOLATION = cv2.INTER_CUBIC
SCALE = 4
SIZE_CONV = 7

In [110]:
# dnn configuration for srcnn
from network.srcnn import srcnn
dnn = srcnn()
dnn.layers = 4
dnn.filter = [128, 64, 64, 1]
dnn.kernelSize =  [(11,11), (3,3), (3,3), (5,5)]
dnn.in_train = False

In [111]:
tick1 = cv2.getTickCount()

In [112]:
model = dnn.network()
model.load_weights(FILEPATH_WEIGHT + FILENAME_WEIGHT)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_89 (Conv2D)           (None, None, None, 128)   15616     
_________________________________________________________________
conv2d_90 (Conv2D)           (None, None, None, 64)    73792     
_________________________________________________________________
conv2d_91 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
conv2d_92 (Conv2D)           (None, None, None, 1)     1601      
Total params: 127,937
Trainable params: 127,937
Non-trainable params: 0
_________________________________________________________________
None


In [113]:
img = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
shape = img.shape

Y_img = cv2.resize(img[:, :, 0], (int(shape[1] * SCALE), int(shape[0] * SCALE)), INTERPOLATION)
img = cv2.resize(img, (int(shape[1] * SCALE), int(shape[0] * SCALE)), INTERPOLATION)

#Y_img = cv2.resize(img[:, :, 0], (int(shape[1] / sizeMux), int(shape[0] / sizeMux)), cv2.INTER_CUBIC)
#Y_img = cv2.resize(Y_img, (shape[1], shape[0]), cv2.INTER_CUBIC)
#img[:, :, 0] = Y_img

img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
cv2.imwrite(OPENCV_NAME, img)

True

In [114]:
Y = numpy.zeros((1, img.shape[0], img.shape[1], 1), dtype=float)
Y[0, :, :, 0] = Y_img.astype(float) / 255.

pre = model.predict(Y, batch_size=1) * 255.
pre[pre[:] > 255] = 255
pre[pre[:] < 0] = 0

pre = pre.astype(numpy.uint8)
img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

if SIZE_CONV > 0:
    img[SIZE_CONV: -SIZE_CONV, SIZE_CONV: -SIZE_CONV, 0] = pre[0, :, :, 0]
else:
    img[2:-2, 2:-2, 0] = pre[0, :, :, 0]

img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
cv2.imwrite(SRCNN_NAME, img)

True

In [117]:
# display image
im1 = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
im2 = cv2.imread(OPENCV_NAME, cv2.IMREAD_COLOR)
im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
im3 = cv2.imread(SRCNN_NAME, cv2.IMREAD_COLOR)
im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2RGB)

pltShow = False

if pltShow:
    plt.figure(num='srcnn',figsize=(16,16))

    plt.subplot(1,3,1)
    plt.title('origin image')
    plt.imshow(im1)

    plt.subplot(1,3,2)
    plt.title('OpenCV')
    plt.imshow(im2)

    plt.subplot(1,3,3)
    plt.title('SRCNN')
    plt.imshow(im3)

In [118]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("processed time: " + str(tick) + " ms")

processed time: 14 mins 24 secs 340 ms
